In [9]:
from __future__ import annotations

```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{Boring Object Orientation}
\author{Moshe Zadka -- https://cobordism.com}
\date{2021}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## OO Python

### Python and object  programming

```{=latex}
\begin{frame}[fragile]
\frametitle{Python and object oriented programming}

Everything is an object
\end{frame}
```

### Why OO design principles

```{=latex}
\begin{frame}[fragile]
\frametitle{Why OO design principles?}

Guidelines to code that is easy to maintain
\end{frame}
```

### Do OO design principles work?

```{=latex}
\begin{frame}[fragile]
\frametitle{Do OO design principles work?}

Yes
\pause
...but
\end{frame}
```


### Principles

```{=latex}
\begin{frame}[fragile]
\frametitle{Principles}

Make your objects more boring!
The simple tricks that they don't want you to know!

\begin{itemize}
\item Declare interfaces\pause 
\item Simplify initialization\pause
\item Avoid mutation\pause
\item Avoid hiding\pause
\item Avoid methods\pause
\item Avoid inheritance
\end{itemize}

\end{frame}


## Interfaces

### Why declare interfaces

```{=latex}

\begin{frame}[fragile]
\frametitle{Why declare interfaces?}

Explicit is better than implicit
\end{frame}
```

### Declaring interfaces with zope.interface

```{=latex}
\begin{frame}[fragile]
\frametitle{Declaring interfaces with zope.interface}
```

In [10]:
from zope import interface

class ISprite(interface.Interface):

    bounding_box = interface.Attribute(
        "The bounding box"
    )

    def intersects(box):
        "Does this intersect with a box"

```{=latex}
\end{frame}
```

### Testing for interface provision

```{=latex}
\begin{frame}[fragile]
\frametitle{Testing for interface provision}
```

In [11]:
from zope.interface import verify

def test_implementation():
    sprite = make_sprite()
    verify.verifyObject(ISprite, sprite)

```{=latex}
\end{frame}
```

## Boring constructors

        
### Interesting constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Interesting constructors}
```

In [12]:
class Stuff:

    def __init__(self, fname):
        # Create a new object
        self.destination = Destination()
        # Call a system call
        self.finput = open(fname)

```{=latex}
\end{frame}
```

### Boring constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Boring constructors}
```

In [13]:
class Stuff:

    def __init__(self, finput, destination):
        self.destination = destination
        self.finput = finput

    @classmethod
    def from_name(cls, name):
        # Create a new object
        destination = Destination()
        # Call a system call
        finput = open(fname)
        return cls(finput, destination)

```{=latex}
\end{frame}
```

### Why boring constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Why boring constructors?}

\begin{itemize}
\item Testing\pause 
\item Correctness\pause
\item Async\pause
\end{itemize}

\end{frame}
```

### Using attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Using attrs}
```

In [14]:
import attr

@attr.s(auto_attribs=True)
class Stuff:
    finput: Any
    destination: Any

```{=latex}
\end{frame}
```

(Finish immutability and methods for next time)

## Immutability

### Immutable objects
    
### Immutablity as bug avoidance
    
### Immutablity as interface simplifying
    
### Frozen attrs

## Methods

### Private methods issues
### Refactoring private methods away
### Polyomorphism
### Bloating
### Singledispatch

## Inheritance-as-API

### Examples in the wild: Twisted
### Examples in the wild: Django
### Examples in the wild: Jupyter
### Issues
### Composition

## Wrap-up

### No rules

### Opaque lessons

### Principles

```{=latex}
\end{document}
```